##  三菱 SLMP 通訊協定

In [ ]:
using System;
using System.Collections.Generic;
using System.Net.Sockets;
using System.IO;

In [ ]:
public class PLC
{
    private TcpClient Client = new TcpClient();
    private NetworkStream Stream {get;set;}
    private string subheader = "5000";
    private string network_number = "00";
    private string destination_station = "FF";
    private string destination_IO_number = "03FF";
    private string request_multi_drop = "00";
    private string reserved_data = "0000";
    private string command ="0401";
    private string subcommand = "0000";
    private string device_code = "D*";
    private string head_device = "000000";
    private string number_device ="0000";
    private string request_data_length = "0000";
    private string value_device = "";

    public void TcpConnect(string HostName, int PortNumber)
    {
        TcpClient c = Client;
        if (!c.Connected) {
            // Keep Alive機能の実装
            var ka = new List<byte>(sizeof(uint) * 3);
            ka.AddRange(BitConverter.GetBytes(1u));
            ka.AddRange(BitConverter.GetBytes(45000u));
            ka.AddRange(BitConverter.GetBytes(5000u));
            c.Client.IOControl(IOControlCode.KeepAliveValues, ka.ToArray(), null);
            try {
                c.Connect(HostName, PortNumber);
                Stream = c.GetStream();
            } catch (SocketException e1) {
                throw new Exception("PLC連線（" + HostName + ":" + PortNumber + "）失敗！", e1);
            }
        }
    }
    public void TcpDisconnect()
    {
        TcpClient c = Client;
                if (c.Connected) {
                    c.Close();
                }
    }
    public string CreateMessage()
    {
        var str2 = reserved_data + command + subcommand + device_code + head_device + number_device + value_device;
        request_data_length = String.Format("{0:X4}", str2.Count());
        var str1 = subheader + network_number + destination_station + destination_IO_number + request_multi_drop + request_data_length + str2;
        return str1;
    }
    byte[] Execute(byte[] iCommand)
    {
        NetworkStream ns = Stream;
        ns.Write(iCommand, 0, iCommand.Length);
        ns.Flush();

        using (var ms = new MemoryStream())
        {
            var buff = new byte[256];
            do
            {
                int sz = ns.Read(buff, 0, buff.Length);
                if (sz == 0)
                {
                    throw new Exception("斷線了");
                }
                ms.Write(buff, 0, sz);
            } while (ns.DataAvailable);
            return ms.ToArray();
        }
    }
    public string Execute(string command)
    {
        var iCommand = Encoding.ASCII.GetBytes(command);
        var res = Execute(iCommand);
        return Encoding.ASCII.GetString(res);
    }
    public string SendMessage(string msg)
    {
        var res = "";
        while (res == "")
        {
            try
            {
                res = Execute(msg);
            }
            catch(Exception e)
            {
                continue;
            }
        }

        if (res.Count()<4)
            return res;
        else if(res.Substring(0,4) == "D000")
        {
            if(res.Count()<18)
                return res;
            else if(res.Substring(18,4) == "0000")
            {
                return res.Substring(22);
            }
            else
            {
                return res.Substring(18,4);
            }
        }
        else
        {
            return "Error";
        }
    }

    public string ReadM(int start_digit,int device_num = 1)
    {
        command = "0401";
        device_code = "M*";
        head_device = String.Format("{0:d6}",start_digit);
        number_device = String.Format("{0:d4}",device_num);
        value_device = "";
        var msg =CreateMessage();
        var res =SendMessage(msg);
        Console.WriteLine($"M{start_digit}={Int32.Parse(res, System.Globalization.NumberStyles.HexNumber)}");
        return res;
    }
    public string ReadD(int start_digit,int device_num = 1)
    {
        command = "0401";
        device_code = "D*";
        head_device = String.Format("{0:d6}",start_digit);
        number_device = String.Format("{0:d4}",device_num);
        value_device = "";
        var msg =CreateMessage();
        var res =SendMessage(msg);
        Console.WriteLine($"D{start_digit}={Int32.Parse(res, System.Globalization.NumberStyles.HexNumber)}");
        return res;
    }
    public string WriteD(int start_digit,int value,int device_num = 1)
    {
        command = "1401";
        device_code = "D*";
        head_device = String.Format("{0:d6}",start_digit);
        number_device = String.Format("{0:d4}",device_num);
        value_device = string.Format("{0:x4}",value);
        var msg =CreateMessage();
        var res =SendMessage(msg);
        Console.WriteLine($"D{start_digit}={value}");
        return res;
    }
    public string WriteM(int start_digit,int value,int device_num = 1)
    {
        command = "1401";
        device_code = "M*";
        head_device = String.Format("{0:d6}",start_digit);
        number_device = String.Format("{0:d4}",device_num);
        value_device = string.Format("{0:x4}",value);
        var msg =CreateMessage();
        var res =SendMessage(msg);
        Console.WriteLine($"M{start_digit}={value}");
        return res;
    }
} 


## 測試 讀取/寫入 M、D 之結果

In [ ]:
var plc = new PLC();
plc.TcpConnect("192.168.3.39",3000);
plc.ReadD(0);
plc.ReadM(1);
plc.WriteD(0,1024);
plc.WriteM(0,0);
plc.TcpDisconnect();
Console.WriteLine("Finish");

D0=1024


M1=0


D0=1024


M0=0


Finish
